In [ ]:
import cv2
import math
import numpy as np
import mediapipe as mp 

In [ ]:
# Parâmetros para para mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Definindo dimensões da janela
w_cam, h_cam = 640, 480

In [ ]:
# Iniciando captura de vídeo
cap = cv2.VideoCapture(0)
cap.set(3, w_cam)
cap.set(4, h_cam)

with mp_hands.Hands(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as hands:

  while cap.isOpened():
    success, image = cap.read()
    
    if not success:
      print("Não foi possível iniciar a WebCam")
      break

    # Invertendo posicionamento horizontal e convertendo BGR para RGB 
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    image_height, image_width, _ = image.shape

    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Cordenadas polegar
        x1 = int(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x * image_width)
        y1 = int(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y * image_height)

        # Cordenadas indicador
        x2 = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width)
        y2 = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height)

        # Centro da reta A -> B
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        
        # Plotando pontos e reta, entre indicador e polegar
        cv2.circle(image, (x1, y1), 7, (255, 0, 0), cv2.FILLED)
        cv2.circle(image, (x2, y2), 7, (255, 0, 0), cv2.FILLED)
        cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 3)
        cv2.circle(image, (cx, cy), 7, (255, 0, 0), cv2.FILLED)

        # Mapeando valores
        comp = math.hypot(x2 - x1, y2 - y1)
        val_bar = np.interp(comp, [50, 350], [400, 150])  
        val_per = np.interp(comp, [50, 350], [0, 100])

        # Plotanto barra de peenchimento e percentual 
        cv2.rectangle(image, (50, 150), (60, 400), (255, 0, 0), 2)
        cv2.rectangle(image, (50, int(val_bar)), (60, 400), (255, 0, 0), cv2.FILLED)
        cv2.putText(image, f"{int(val_per)} %", (40, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)

    # Texto da tela
    cv2.putText(image, 'HandTalk', (40, 50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
    # Exibe vídeo 
    cv2.imshow('HandTalk', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

cap.release()
cv2.destroyAllWindows()